In [ ]:
from bs4 import BeautifulSoup
import urllib.request
import pandas as pd
import html5lib 
from lxml import html
import numpy as np
from helpers import *
#import helpers as helpers

In [ ]:
# The URL of the "start" of the IS-Academia page listing the students.
home_url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.filter?ww_x_GPS=-1&ww_i_reportModel=133685247"

# The general form of the URL, with the fields to be replaced later (e.g. [UNITE_ACADEMIQUE])
base_url = "http://isa.epfl.ch/imoniteur_ISAP/!GEDPUBLICREPORTS.html?ww_x_GPS=-1&ww_i_reportModel=133685247&ww_i_reportModelXsl=133685270&ww_x_UNITE_ACAD=[UNITE_ACADEMIQUE]&ww_x_PERIODE_ACAD=[PERIODE_ACADEMIQUE]&ww_x_PERIODE_PEDAGO=[PERIODE_PEDAGOGIQUE]&ww_x_HIVERETE=null"

# Get the raw content from the page
with urllib.request.urlopen(home_url) as url:
    s = url.read()

soup = BeautifulSoup(s, 'html.parser')

# Parse the content
url_section = 0
url_years = {}
url_master = {}

#For loop on all the possible fields with <option> to get the value associated with each year, or each "Bachelor" semester
for link in soup.find_all('option'):
    if link.text == 'Informatique':
        url_section = link.get('value')
    if "20" in link.text:
        #print(link.text)
        url_years[link.text] = link.get('value')
    if 'Master' in link.text or 'Mineur' in link.text:
        url_master[link.text] = link.get('value')

        
print(url_years)
print(url_master)
#frame= frame.drop([2,3,4,5,6,8,9,11],axis=1)
#frame.head(10)

In [ ]:
def getFullUrl(PeriodeAcad, PeriodePedag):
    url = base_url
    url = url.replace('[UNITE_ACADEMIQUE]', str(url_section))
    url = url.replace('[PERIODE_ACADEMIQUE]', str(url_years[PeriodeAcad]))
    url = url.replace('[PERIODE_PEDAGOGIQUE]', str(url_master[PeriodePedag]))
    return url

Assumptions : 
- All students start in MS1 and do it only once since afterward it will be MS3.

In [ ]:
def MasterStart(year_string,master_student,pedag) : 
    current_url = getFullUrl(year_string, pedag)
    with urllib.request.urlopen(current_url) as url:
        html = url.read()
    soup = BeautifulSoup(html, 'html.parser')
    master_frame = pd.read_html(soup.prettify(), header=1)
    if  len(master_frame)>0 :
        master_frame = master_frame[0].drop(0,axis=0)
        master_frame_sciper = master_frame[['No Sciper']]
        master_frame = master_frame.set_index('No Sciper')
        for sciper in master_frame_sciper.values:
            sciper = sciper[0]
            
            if pedag == 'Master semestre 1' or sciper not in master_student.index:
             master_student.loc[sciper] = 0   
             master_student.loc[sciper,pedag] = 1
            else :
             master_student.loc[sciper,pedag] += 1   
            master_student.loc[sciper,'Civilité'] = master_frame.loc[sciper,'Civilité']
            master_student.loc[sciper,'Nom Prénom'] = master_frame.loc[sciper,'Nom Prénom']
            master_student.loc[sciper,'Spécialisation'] = master_frame.loc[sciper,'Spécialisation']
            master_student.loc[sciper,'Mineur'] = master_frame.loc[sciper,'Mineur']
    return master_student

In [ ]:
master_student = pd.DataFrame(columns = ['Civilité', 'Nom Prénom', 
                                         'Master semestre 1', 'Master semestre 2', 'Master semestre 3',
                                         'Projet Master printemps', 'Projet Master automne', 
                                         'Spécialisation', 'Mineur'])
master_student.index.name = 'No Sciper'

pedagogic_year = ['Master semestre 1','Master semestre 2','Master semestre 3',
                  'Projet Master printemps','Projet Master automne']
for year in range(2007,2016):
    year_string = str(year) + '-' + str(year+1)
    for pedag in  pedagogic_year :
     master_student = MasterStart(year_string,master_student,pedag)


.Assumptions  : 
 - possible to have project master in spring and autumn
 - if no project master register assume thta student had project outside epfl and add +1 semester

In [ ]:
print(master_student.index.is_unique)
## sum over all the master 1,2,3 plus additional master project.
time = (master_student['Master semestre 1']+master_student['Master semestre 2']+
        master_student['Master semestre 3']+master_student['Projet Master printemps']+
        master_student['Projet Master automne']+
        1-(master_student['Projet Master printemps']+ master_student['Projet Master automne'])%1)

master_student['Time to Finish']= time
master_student.head()

In [ ]:
import scipy.stats as stats
specialisation = master_student[master_student['Spécialisation']==master_student['Spécialisation']]                        
mean_time_specialisation = specialisation['Time to Finish']
mean_time  = master_student['Time to Finish'].mean()
print(mean_time)
print(mean_time_specialisation.mean())
stats.ttest_1samp(a= mean_time_specialisation, 
                 popmean= mean_time)

In [ ]:
master_student.describe()

In [ ]:
master_student_grouped = master_student.groupby(master_student.Spécialisation)
mean_spec = pd.DataFrame(columns=['Mean'])

for spec,student in master_student_grouped:
    #print('spec', spec)
    #print('student', student['Time to Finish'])
    mean_spec.loc[spec] = np.mean(student['Time to Finish'].values)

mean_spec

In [ ]:
#print(master_student.head(20))
#master_student = master_student.drop('Mineur', axis=1)

master_student_with_Spec = master_student[master_student.Spécialisation == master_student.Spécialisation]
list_Spec = master_student_with_Spec['Spécialisation'].drop_duplicates().values

#print(master_student_with_Spec.shape)
#print(master_student_with_Spec.sort_index().head())

#baseball_newind.sort_index(axis=1).head()
master_student_with_Spec['No Sciper'] = master_student_with_Spec.index
master_student_with_Spec = master_student_with_Spec.set_index(['Spécialisation'])
master_student_with_Spec = master_student_with_Spec.sort_index()
master_student_with_Spec = master_student_with_Spec.set_index([master_student_with_Spec.index, 'No Sciper'])

master_student_with_Spec.head(100)

#print(master_student_with_Spec.head())


In [ ]:
#segments.groupby(segments.name).seg_length.sum().sort_values(ascending=False, inplace=False).head(20)

master_student_with_Spec.groupby(master_student_with_Spec.index).sum()

In [ ]:
#master_student['Spécialisation'].drop_duplicates()
print(list_Spec)
master_student_with_Spec.loc[list_Spec[1]]

In [ ]:

for spec in list_Spec:
    master_student_in_spec = master_student_with_Spec.loc[spec]
    print('Specialisation ' + spec)
    print(master_student_in_spec.loc[:,'Time to Finish'].shape)
    print(stats.ttest_1samp(a= np.array(master_student_in_spec.loc[:,'Time to Finish']), popmean=mean_time))
    print()
    
# Note : we get a NaN pvalue when there is only 1 student in a Specialisation as there is not enough data to perform
# a statistical test.

We observe that there is a statistically significant difference of Master duration for students choosing Specifications **Internet Computing**, **Foundations of Software**, **Signals, Images and Interfaces**, **Computer Engineering - SP**, **Computer Engineering - SP** and **Data Analytics**